## Convenient Packing of List Columns into Nested Columns

In [1]:
import nested_pandas as npd
from nested_pandas import NestedFrame
from nested_pandas.series.packer import pack_lists

expected = NestedFrame(
    {"a": [1, 2, 3], "b": [[1, 2, 3], [4, 5, 6], [7, 8, 9]], "c": [[10, 20, 30], [40, 50, 60], [70, 80, 90]]}
)
ndf = expected.copy()
ndf

,a,b,c
0,1,"[1, 2, 3]","[10, 20, 30]"
1,2,"[4, 5, 6]","[40, 50, 60]"
2,3,"[7, 8, 9]","[70, 80, 90]"


In [2]:
packed_expected = pack_lists(ndf[["b", "c"]], name="nested")
packed_expected

0    [{b: 1, c: 10}; …] (3 rows)
1    [{b: 4, c: 40}; …] (3 rows)
2    [{b: 7, c: 70}; …] (3 rows)
Name: nested, dtype: nested<b: [int64], c: [int64]>

In [3]:
# join back to df as nested col
expected = expected.join(packed_expected, how="left")
expected

a          b             c                       nested
0  1  [1, 2, 3]  [10, 20, 30]  [{b: 1, c: 10}; …] (3 rows)
1  2  [4, 5, 6]  [40, 50, 60]  [{b: 4, c: 40}; …] (3 rows)
2  3  [7, 8, 9]  [70, 80, 90]  [{b: 7, c: 70}; …] (3 rows)

In [4]:
ndf

,a,b,c
0,1,"[1, 2, 3]","[10, 20, 30]"
1,2,"[4, 5, 6]","[40, 50, 60]"
2,3,"[7, 8, 9]","[70, 80, 90]"


In [5]:
ndf2 = ndf.copy()

In [6]:
# Now let's pack the lists using a convenience method
ndf = ndf.nest_lists(columns=["b", "c"], name="nested")
ndf

a                       nested
0  1  [{b: 1, c: 10}; …] (3 rows)
1  2  [{b: 4, c: 40}; …] (3 rows)
2  3  [{b: 7, c: 70}; …] (3 rows)

In [7]:
ndf.nested.nest.to_flat()

,b,c
0,1,10
0,2,20
0,3,30
1,4,40
1,5,50
1,6,60
2,7,70
2,8,80
2,9,90


In [8]:
# However we can also use a static constructor
ndf2 = NestedFrame.from_lists(ndf2, list_columns=["b", "c"], name="nested")
ndf2

a                       nested
0  1  [{b: 1, c: 10}; …] (3 rows)
1  2  [{b: 4, c: 40}; …] (3 rows)
2  3  [{b: 7, c: 70}; …] (3 rows)

In [9]:
ndf2.nested.nest.to_flat()

,b,c
0,1,10
0,2,20
0,3,30
1,4,40
1,5,50
1,6,60
2,7,70
2,8,80
2,9,90


## Error checking

In [10]:
# Checks that we raise a more helpful exception for non-list columns
expected.copy().nest_lists(columns=["b", "a"], name="nested")

ValueError: Cannot pack column a which does not contain an iterable list based on its first value, 1.